In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score as cv_score
from sklearn.decomposition import PCA, FastICA
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics 
from sklearn.ensemble import RandomForestRegressor
import datetime
import copy
%matplotlib inline

/home/mikhail/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv', parse_dates=['timestamp'])
test = pd.read_csv('test.csv', parse_dates=['timestamp'])
id_test = test.id

bad_index = train[train.life_sq > train.full_sq].index
train.loc[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.loc[equal_index, "life_sq"] = test.loc[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.loc[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.loc[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.loc[kitch_is_build_year, "build_year"] = train.loc[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.loc[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
train.loc[bad_index, "build_year"] = np.NaN
bad_index = test[test.build_year < 1500].index
test.loc[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index
train.loc[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index
test.loc[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.loc[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.loc[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.loc[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.loc[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.loc[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.loc[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.loc[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.loc[bad_index, "state"] = np.NaN
test.state.value_counts()

train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

In [3]:
# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek

# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)

test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train['apartment_name']=train.sub_area + train['metro_km_avto'].astype(str)
test['apartment_name'] =test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)


In [4]:
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
#x_test = test.drop(["id", "timestamp", "average_q_price"], axis=1)
y_train = train["price_doc"]
x_test = test.drop(["id", "timestamp"], axis=1)

num_train = len(x_train)
x_all = pd.concat([x_train, x_test])

for c in x_all.columns:
    if x_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_all[c].values))
        x_all[c] = lbl.transform(list(x_all[c].values))

x_train = x_all[:num_train]
x_test = x_all[num_train:]


# 1

In [5]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}


In [20]:
dtrain = xgb.DMatrix(x_train, (y_train))
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=10, show_stdv=True)

[0]	train-rmse:2541.01+3.65795	test-rmse:2541.41+7.67968
[10]	train-rmse:1574.64+2.19561	test-rmse:1579.74+6.51581
[20]	train-rmse:1016.85+1.96	test-rmse:1029.35+5.99849
[30]	train-rmse:709.071+1.45778	test-rmse:733.36+5.91882
[40]	train-rmse:550.94+1.90094	test-rmse:587.42+5.44683
[50]	train-rmse:475.398+2.06092	test-rmse:522.244+5.40138
[60]	train-rmse:439.781+2.15244	test-rmse:495.045+5.50645
[70]	train-rmse:421.605+2.80307	test-rmse:482.782+5.06266
[80]	train-rmse:410.557+2.93475	test-rmse:477.619+5.05179
[90]	train-rmse:402.873+2.63151	test-rmse:474.594+4.91194
[100]	train-rmse:396.779+2.20534	test-rmse:472.866+5.0862
[110]	train-rmse:391.633+2.42505	test-rmse:471.871+5.36626
[120]	train-rmse:386.77+2.26003	test-rmse:471.323+5.33668
[130]	train-rmse:382.417+2.11722	test-rmse:470.746+5.17933
[140]	train-rmse:378.5+2.17218	test-rmse:470.18+4.96584
[150]	train-rmse:374.476+2.66794	test-rmse:469.761+5.12273
[160]	train-rmse:370.545+2.33124	test-rmse:469.341+4.94998
[170]	train-rmse:36

In [22]:
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 330#422
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

y_predict = model.predict(dtest)

In [23]:
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.to_csv('model1.csv', index=None)
output.head()

,id,price_doc
0,30474,5242948.5
1,30475,7827290.5
2,30476,5219908.5
3,30477,6075087.5
4,30478,4779251.5


# 2

In [16]:
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.04,
    'verbose': 10,
    'num_leaves': 32,
    'bagging_fraction': 0.99,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'max_bin': 500,
    'max_depth': 10,
}

In [17]:
ltrain = lgb.Dataset(x_train, (y_train)/1e6)
cv_output = lgb.cv(lgb_params, ltrain, num_boost_round=1000, nfold = 3, early_stopping_rounds=50,
    verbose_eval=10, show_stdv=True)

[10]	cv_agg's rmse: 0.533207 + 0.00449067
[20]	cv_agg's rmse: 0.498594 + 0.00510635
[30]	cv_agg's rmse: 0.481379 + 0.00555389
[40]	cv_agg's rmse: 0.47151 + 0.00580002
[50]	cv_agg's rmse: 0.465594 + 0.00605258
[60]	cv_agg's rmse: 0.462096 + 0.0063232
[70]	cv_agg's rmse: 0.459646 + 0.00639501
[80]	cv_agg's rmse: 0.457666 + 0.006386
[90]	cv_agg's rmse: 0.456146 + 0.00635149
[100]	cv_agg's rmse: 0.454918 + 0.00634885
[110]	cv_agg's rmse: 0.454041 + 0.00628218
[120]	cv_agg's rmse: 0.453377 + 0.00630741
[130]	cv_agg's rmse: 0.452904 + 0.00650133
[140]	cv_agg's rmse: 0.452551 + 0.0064866
[150]	cv_agg's rmse: 0.452107 + 0.00640434
[160]	cv_agg's rmse: 0.451964 + 0.00643965
[170]	cv_agg's rmse: 0.451829 + 0.00653078
[180]	cv_agg's rmse: 0.451706 + 0.0065536
[190]	cv_agg's rmse: 0.451439 + 0.00651644
[200]	cv_agg's rmse: 0.451374 + 0.00666761
[210]	cv_agg's rmse: 0.451318 + 0.00662204
[220]	cv_agg's rmse: 0.451194 + 0.0065602
[230]	cv_agg's rmse: 0.451105 + 0.00655291
[240]	cv_agg's rmse: 0.4510

In [18]:
ltrain = lgb.Dataset(x_train, y_train/1e6)
num_boost_round = 600

model = lgb.train(lgb_params, ltrain, num_boost_round=num_boost_round)

y_predict_lgb = model.predict(x_test)*1e6

In [19]:
output_lgb = pd.DataFrame({'id': id_test, 'price_doc': y_predict_lgb})
output_lgb.to_csv('model1_lgb_log.csv', index=None)
output_lgb.head()

,id,price_doc
0,30474,5.204563e+06
1,30475,7.914491e+06
2,30476,4.635354e+06
3,30477,6.174865e+06
4,30478,4.813138e+06


# 3

In [18]:
rf = RandomForestRegressor(n_estimators=500, max_features=120, n_jobs=-1)

x_train_rf = x_train.fillna(x_train.median())
x_test_rf  = x_test.fillna(x_train.median())

rf.fit(x_train_rf, y_train)

y_predict_rf = rf.predict(x_test_rf)

In [19]:
output_rf = pd.DataFrame({'id': id_test, 'price_doc': y_predict_rf})
output_rf.to_csv('model1_rf.csv', index=None)
output_rf.head()

,id,price_doc
0,30474,5435984.446
1,30475,8353995.906
2,30476,5344476.910
3,30477,5943234.946
4,30478,4997280.254
